<a href="https://colab.research.google.com/github/marquesarthur/vanilla-bert-vs-huggingface/blob/main/vanilla_keras_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://colab.research.google.com/drive/14b2rbIgwhQ1BI-zkyiMjQv-jV85xj9tf#scrollTo=5qSd2lLwJ7lH


**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between Y and sentences x ∈ X 
*  **The model must determine if x is relevant to task Y**

In [1]:
# @title Install dependencies

# !pip install -q keras-bert==0.85.0 keras-rectified-adam==0.15.0
!pip install -q keras-bert keras-rectified-adam
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install -q scikit-learn tqdm pandas python-Levenshtein path colorama

     |████████████████████████████████| 50 kB 2.5 MB/s 


In [3]:
# @title Download git repo
!git clone https://github.com/marquesarthur/vanilla-bert-vs-huggingface.git

Cloning into 'vanilla-bert-vs-huggingface'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (541/541), done.
remote: Compressing objects: 100% (478/478), done.
remote: Total 541 (delta 56), reused 537 (delta 55), pack-reused 0
Receiving objects: 100% (541/541), 7.79 MiB | 7.08 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [4]:
%cd vanilla-bert-vs-huggingface
!git pull
!ls -l

/content/vanilla-bert-vs-huggingface
Already up to date.
total 5236
-rw-r--r-- 1 root root    7988 Sep  6 17:58 ds_android.py
drwxr-xr-x 7 root root    4096 Sep  6 17:58 expert_answers
drwxr-xr-x 5 root root    4096 Sep  6 17:58 expert_tasks
drwxr-xr-x 2 root root    4096 Sep  6 17:58 hugging
-rw-r--r-- 1 root root     356 Sep  6 17:58 README.md
-rw-r--r-- 1 root root 5331079 Sep  6 17:58 relevance_corpus.json
drwxr-xr-x 2 root root    4096 Sep  6 17:58 vanilla


In [5]:
# @title Download BERT model
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [6]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

4 12  https://stackoverflow.com/questions/33241952
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
5 47  https://developer.android.com/reference/android/widget/ArrayAdapter
9 21  https://stackoverflow.com/questions/6442054
3 22  https://github.com/nostra13/Android-Universal-Image-Loader/issues/462
22 211  https://www.raywenderlich.com/155-android-listview-tutorial-with-kotlin
21 59  https://guides.codepath.com/android/Using-an-ArrayAdapter-with-ListView
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/javase/8/javafx/layout-tutorial/size_align.htm
39 129  

In [7]:
from collections import Counter, defaultdict

cnt = Counter([d['category_index'] for d in raw_data])

total = sum(cnt.values())

labels_cnt = [cnt[0] / float(total), cnt[1] / float(total)]
print('label distribution')
print('')
print('not-relevant -- {:.0f}%'.format(labels_cnt[0] * 100))
print('RELEVANT ------ {:.0f}%'.format(labels_cnt[1] * 100))

label distribution

not-relevant -- 88%
RELEVANT ------ 12%


In [8]:
# @title Set environment variables

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver)

# sklearn libs
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

# Tensorflow Imports
import tensorflow as tf
from tensorflow.python import keras
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam


# Keras-bert imports
from keras_radam import RAdam
from keras_bert import Tokenizer
from keras_bert import get_custom_objects
from keras_bert import load_trained_model_from_checkpoint

os.environ['TF_KERAS'] = '1'

# Bert Model Constants
SEQ_LEN = 128
BATCH_SIZE = 32 # larger batch size causes OOM errors
EPOCHS = 3
LR = 2e-5

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

Creating converter from 7 to 5
Creating converter from 5 to 7
Creating converter from 7 to 5
Creating converter from 5 to 7


In [9]:
# @title Initialize Variables

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [10]:
# @title JSON to dataframe helper functions
def undersample_df(df, n_times=3):
    class_0,class_1 = df.category_index.value_counts()
    c0 = df[df['category_index'] == 0]
    c1 = df[df['category_index'] == 1]
    df_0 = c0.sample(int(n_times * class_1))
    
    undersampled_df = pd.concat([df_0, c1],axis=0)
    return undersampled_df

def get_ds_synthetic_data(min_w=3):
  short_task = {
      "bugzilla": """How to query bugs using the custom fields with the Bugzilla REST API?""",
      "databases": """Which technology should be adopted for the database layer abstraction: Object/Relational Mapping (ORM) or a Java Database Connectivity API (JDBC)?""",
      "gpmdpu": """Can I bind the cmd key to the GPMDPU shortcuts?""",
      "lucene": """How does Lucene compute similarity scores for the BM25 similarity?""",
      "networking": """Which technology should be adopted for the notification system, Server-Sent Events (SSE) or WebSockets?""",
  }

  with open('relevance_corpus.json') as ipf:
      aux = json.load(ipf)
      raw_data = defaultdict(list)
      for d in aux:
          if d['task'] == 'yargs':
              continue

          raw_data['text'].append(d['text'])
          raw_data['question'].append(short_task[d['task']])
          raw_data['source'].append(d['source'])
          raw_data['category_index'].append(1 if d['weight'] > min_w else 0)
          raw_data['weights'].append(d['weight'] if d['weight'] > min_w else 0)

      data = pd.DataFrame.from_dict(raw_data)
      data = undersample_df(data, n_times=1)
      data = data.sample(frac=1).reset_index(drop=True)
      
  return data

def get_class_weights(y, smooth_factor=0, upper_bound=5.0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    clazz = {cls: float(majority / count) for cls, count in counter.items()}
    result = {}
    for key, value in clazz.items():
        if value > upper_bound:
            value = upper_bound
        
        result[key] = value
    return result

def add_raw_data(result, data):
    result['text'].append(data['text'])
    result['question'].append(data['question'])
    result['source'].append(data['source'])
    result['category_index'].append(data['category_index'])
    result['weights'].append(data['weights'])


In [11]:
# @title Tokenizer

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

tokenizer = Tokenizer(token_dict)

# FIXME: global variable that is referenced inside the train/test functions...
model = None

In [12]:
# @title data encoder

def encode_data(df, tokenizer, over_sampling=1, testing=False):
    relevant = 1
    indices, segments, labels, metadata = [], [], [], []
    
    for index, row in df.iterrows():
        _ids, _segments = tokenizer.encode(
            first=row["text"], 
            second=row["question"], 
            max_len=SEQ_LEN
        )
        
        label = row["category_index"]
        if label == relevant:
            for _ in range(over_sampling):
                indices.append(_ids)
                segments.append(_segments)
                labels.append(label)
                metadata.append((row['weights'], row['text'], row["question"]))
        else:
            indices.append(_ids)
            segments.append(_segments)
            labels.append(label)
            metadata.append((row['weights'], row['text'], row["question"]))
        
    # zip data into single list, shuffle everything and decompress
    items = list(zip(indices, segments, labels, metadata))
    np.random.shuffle(items)
    indices, segments, labels, metadata = zip(*items)
    indices = np.array(indices)
    
    # checks if array size is equals to batch size. If it's not, remove the last n entries to make it divisable
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0 and not testing:
        indices, segments, labels, metadata = indices[:-mod], segments[:-mod], labels[:-mod], metadata[:-mod]
    
    X, y = [indices, np.array(segments)], np.array(labels)

    return X, y, metadata

In [13]:
# @title Metrics & Logging functions

from sklearn.metrics import classification_report

recommendation_metrics = defaultdict(list)
prediction_metrics = defaultdict(list)

classification_report_lst = []
log_examples_lst = []

def aggregate_macro_metrics(store_at, precision, recall, fscore):   
    store_at['precision'].append(precision)
    store_at['recall'].append(recall)
    store_at['fscore'].append(fscore)

def aggregate_recommendation_metrics(store_at, k, precision_at_k, pyramid_precision_at_k):
    store_at['k'].append(k)
    store_at['precision'].append(precision_at_k)
    store_at['∆ precision'].append(pyramid_precision_at_k)

def log_examples(task_title, source, text, pweights, y_predict, y_probs, k=10):
    # get the predicted prob at every index
    idx_probs = [(idx, y_predict[idx], y_probs[idx]) for idx, _ in enumerate(y_predict)]
    
    # filter probs for all indexes predicted as relevant  
    idx_probs = list(filter(lambda k: k[1] == 1, idx_probs))
    
    most_probable = sorted(idx_probs, key=lambda i: i[2], reverse=True)
    
    result = [idx for idx, _, _ in most_probable][:k]
    
    for idx in result:
        log_examples_lst.append((
            source, 
            task_title,
            pweights[idx],
            y_predict[idx],
            y_probs[idx],
            text[idx]
        ))

def _precision_at_k(y_test, y_predict, y_prob, k=10):
    # get the predicted prob at every index
    idx_probs = [(idx, y_predict[idx], y_prob[idx]) for idx, _ in enumerate(y_test)]
    
    # filter probs for all indexes predicted as relevant  
    idx_probs = list(filter(lambda k: k[1] == 1, idx_probs))
    
    most_probable = sorted(idx_probs, key=lambda i: i[2], reverse=True)
    result = [y_test[idx] * y_predict[idx] for idx, _, _ in most_probable]   
    y_predict = [y for _, y, _ in most_probable]
    
    result = result[:k]
    y_predict = y_predict[:k]
    ratio = sum(result) / float(len(y_predict) + 0.00001)
    return ratio     


def _pyramid_score(y_optimal, y_predicted, y_prob, k=10):

    # create reference table for weights 
    # y_predicted = [i for i in y_optimal]
    # get the predicted prob at every index
    idx_probs = [(idx, y_optimal[idx], y_predicted[idx], y_prob[idx]) for idx, _ in enumerate(y_optimal)]
    
    # filter probs for all indexes predicted as relevant  
    idx_probs = list(filter(lambda aux: aux[2] == 1, idx_probs))

    # sort
    most_probable = sorted(idx_probs, key=lambda i: i[3], reverse=True)

    # compute predicted and optimal score up until K
    predicted_score = [w for _, w, _, _ in most_probable][:k]
    optimal_score = sorted(y_optimal, reverse=True)[:k]
    
    ratio = sum(predicted_score) / float(sum(optimal_score) + 0.00001)
    return ratio           

In [14]:
#@title Training procedures

def get_train_val_test(task_uid, size=0.9, undersample=False, aug=True, undersample_n=3):
    if not isinstance(task_uid, list):
        task_uid = [task_uid]
        
    train_data_raw = defaultdict(list)
    test_data_raw = defaultdict(list)
    
    for _data in tqdm(CORPUS):
        if _data['question'] in task_uid:
            add_raw_data(test_data_raw, _data)
        else:
            add_raw_data(train_data_raw, _data)
    
    train_val = pd.DataFrame.from_dict(train_data_raw)
    test = pd.DataFrame.from_dict(test_data_raw)
    
    # https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    #  randomize rows....    
    train_val = train_val.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    
    if undersample:
        train_val = undersample_df(train_val, n_times=undersample_n)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
        
    if aug:
        train_val = pd.concat([train_val, get_ds_synthetic_data()],axis=0)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
    
    weights = get_class_weights(train_val['category_index'].tolist())
    
    train, val = train_test_split(
        train_val, 
        stratify=train_val['category_index'].tolist(), 
        train_size=size
    )
    
    return train, val, test, weights        

In [15]:
# @title Testing procedures

def test_model(source, df_test, model, tokenizer):
    
    test_x, test_y, metadata = encode_data(df_test, tokenizer, testing=True)
    
    logger.info(Fore.YELLOW + str(len(test_x)) + Style.RESET_ALL)
    
    text = [m[1] for m in metadata]
    pweights = [m[0] for m in metadata]
    task_title = metadata[0][2]

    predicts = model.predict(test_x, verbose=True)
    
    y_probs = predicts[:, 1]
    y_predict = predicts.argmax(axis=-1)

    accuracy = accuracy_score(test_y, y_predict)
    macro_f1 = f1_score(test_y, y_predict, average='macro')
    
    classification_report_lst.append(classification_report(test_y, y_predict))

    logger.info("-" * 20)    
    
    logger.info("Y")
    logger.info("[0s] {} [1s] {}".format(
        len(list(filter(lambda k: k== 0, test_y))),
        len(list(filter(lambda k: k== 1, test_y)))
    ))
    
        
    logger.info("predicted")
    logger.info("[0s] {} [1s] {}".format(
        len(list(filter(lambda k: k== 0, y_predict))),
        len(list(filter(lambda k: k== 1, y_predict)))
    ))
    
    logger.info("-" * 20)
    
    logger.info("Accuracy: {:.4f}".format(accuracy))
    logger.info("macro_f1: {:.4f}".format(macro_f1))

    precision, recall, fscore, _ = precision_recall_fscore_support(test_y, y_predict, average='macro')
    
    aggregate_macro_metrics(prediction_metrics, precision, recall, fscore)
    
    logger.info("Precision: {:.4f}".format(precision))
    logger.info("Recall: {:.4f}".format(recall))
    logger.info("F1: {:.4f}".format(fscore))
    
    logger.info("-" * 20)
    
    for k in [3, 5, 10]:
        p_at_k = _precision_at_k(test_y, y_predict, y_probs, k=k)
        score_at_k = _pyramid_score(pweights, y_predict, y_probs, k=k)
                                     
        aggregate_recommendation_metrics(recommendation_metrics, k, p_at_k, score_at_k)
        
        logger.info("")
        logger.info("Precision_at_{}: {:.4f}".format(k, p_at_k))
        logger.info("Pyramid_at_{}: {:.4f}".format(k, score_at_k))
    logger.info("-" * 20)
    
    log_examples(task_title, source, text, pweights, y_predict, y_probs, k=5)

In [16]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
random.seed(20210906)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold

n_splits = 10
kf = KFold(n_splits=n_splits)
np_tasks_arr = np.array(all_tasks)

idx_split = 0
for train_index, test_index in kf.split(np_tasks_arr):    
    test_tasks_lst = np_tasks_arr[test_index].tolist()
    
    logger.info("")
    logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
    logger.info('\n'.join(test_tasks_lst))
    
    df_train, df_val, df_test, weights = get_train_val_test(test_tasks_lst, undersample=True, undersample_n=2) 
    
    logger.info('-' * 10)
    logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
    logger.info(str(df_train.category_index.value_counts()))
    logger.info("")

    logger.info(Fore.RED + 'val'+ Style.RESET_ALL)
    logger.info(str(df_val.category_index.value_counts()))
    logger.info("")

    logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
    logger.info(str(df_test.category_index.value_counts()))
    logger.info("")

    logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
    logger.info(str(weights))
    logger.info('-' * 10)
    
    train_x, train_y, _ = encode_data(df_train, tokenizer, over_sampling=1)
    val_x, val_y, _ = encode_data(df_val, tokenizer)
    

    model = load_trained_model_from_checkpoint(
      config_path,
      checkpoint_path,
      training=True,
      trainable=True,
      seq_len=SEQ_LEN
    )
    
    inputs = model.inputs[:2]
    dense = model.get_layer('NSP-Dense').output
    outputs = keras.layers.Dense(units=2, activation='softmax', name="probs")(dense)
    model = keras.models.Model(inputs, outputs)

    optimizer = Adam(lr=LR)

    
    model.compile(
      optimizer=optimizer,
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'],
    )
    
    
    logger.info("")
    logger.info(Fore.RED + f"Training model" + Style.RESET_ALL)
    history = model.fit(
        train_x,
        train_y,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=weights,
        validation_data=(val_x, val_y)
    )
    
    logger.info("")
    logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
    for source in df_test["source"].unique():
        df_source = df_test[df_test["source"] == source]   

        logger.info(source)
        test_model(source, df_source, model, tokenizer)
            
    idx_split += 1
    break


Fold 0
Android: rotate canvas around the center of the screen
Why settings.xml layout is overlapping the ActionBar/Toolbar?
Permission Denial when trying to access contacts in Android
Is there an accepted best-practice on making asynchronous HTTP requests in Android?
Generating an error when using Provider for scoped instances


100%|██████████| 7961/7961 [00:00<00:00, 838839.73it/s]

NumExpr defaulting to 2 threads.
----------
train
0    1595
1     879
Name: category_index, dtype: int64

val
0    177
1     98
Name: category_index, dtype: int64

test
0    966
1    195
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 1.8137154554759467}
----------


From /tensorflow-1.15.2/python3.7/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
From /tensorflow-1.15.2/python3.7/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Entity <bound method TokenEmbedding.call of <keras_bert.layers.embedding.TokenEmbedding object at 0x7f651d783790>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cau

In [17]:
#@title Metrics report
def avg_recommendation_metric_for(data, k=3, filter_outliers=True):
    __precision = []
    __pyramid = []
    
    total_len = len(data['k'])
    
    for idx in range(total_len):
        
        __value = data['k'][idx]
        if __value  == k:
            if filter_outliers:            
                if data['precision'][idx] > 0.:
                    __precision.append(data['precision'][idx])
                if data['∆ precision'][idx] > 0.:
                    __pyramid.append(data['∆ precision'][idx])
            else:
                __precision.append(data['precision'][idx])
                __pyramid.append(data['∆ precision'][idx])
                

    return np.mean(__precision), np.mean(__pyramid)

def avg_macro_metric_for(data):
    __precision = data['precision']
    __recall = data['recall']
    __fscore = data['fscore']

    return np.mean(__precision), np.mean(__recall), np.mean(__fscore)    

In [18]:

_precision, __pyramid_score = avg_recommendation_metric_for(
    recommendation_metrics, 
    k=3
)

logger.info(Fore.YELLOW + "k=3" + Style.RESET_ALL)
logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
logger.info("pyramid:   " + Fore.RED + "{:.3f}".format(__pyramid_score) + Style.RESET_ALL)

k=3
precision: 0.722
pyramid:   0.564


In [19]:
_precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)

logger.info("")
logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


Model metrics
precision: 0.589
recall:    0.648
f1-score:  0.567


In [20]:
def examples_per_source_type(source_type='misc', n_samples=None):
  _sources = list(set([x[0] for x in log_examples_lst]))

  _template = "[w={}]" + Fore.RED + "[y={}]" + Fore.YELLOW + "[p={:.4f}]" + Style.RESET_ALL + " {}"

  idx = 0
  for s in _sources:
      examples_in_source = []
      if source_type == 'api' and ('docs.oracle' in s or 'developer.android' in s):
          examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
          task_title = examples_in_source[0][1]
          idx += 1
      elif source_type == 'so' and ('stackoverflow.com' in s):
          examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
          task_title = examples_in_source[0][1]            
          idx += 1
      elif source_type == 'git' and ('github.com' in s):
          examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
          task_title = examples_in_source[0][1]
          idx += 1
      elif source_type == 'misc' and 'github.com' not in s and 'docs.oracle' not in s and 'developer.android' not in s and 'stackoverflow.com' not in s:
          examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
          task_title = examples_in_source[0][1]
          idx += 1
      if not examples_in_source:
          continue
      logger.info('')
      logger.info(Fore.RED + f"{task_title}" + Style.RESET_ALL)    
      logger.info(s)
      logger.info('')

      for _, _, pweights, y_predict, y_probs, text in examples_in_source:
          logger.info(_template.format(pweights, y_predict, y_probs, text))
          logger.info('')
      logger.info('-' * 20)
      
      if n_samples and idx >= n_samples:
        break
    

In [21]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
examples_per_source_type(source_type='api', n_samples=8)

API

Permission Denial when trying to access contacts in Android
https://developer.android.com/training/permissions/requesting

[w=3][y=1][p=0.7915] At that time, your app can request the runtime permission that's required for accessing that data.

[w=0][y=1][p=0.7840] To request a single permission, use RequestPermission.

[w=1][y=1][p=0.7812] This method returns either PERMISSION_GRANTED or PERMISSION_DENIED, depending on whether your app has the permission.

[w=1][y=1][p=0.7702] Request the runtime permission that your app requires in order to access the private user data.

[w=0][y=1][p=0.7699] You can fulfill many use cases in your app, such as taking photos, pausing media playback, and displaying relevant ads, without needing to declare any permissions.

--------------------

Is there an accepted best-practice on making asynchronous HTTP requests in Android?
https://developer.android.com/training/volley/request

[w=1][y=1][p=0.8696] Specify a URL and receive a raw string in respon

In [22]:
#@title Sample prediction outputs for GIT sources

logger.info(Fore.RED + "GIT" + Style.RESET_ALL)
examples_per_source_type(source_type='git', n_samples=4)

GIT

Generating an error when using Provider for scoped instances
https://github.com/google/dagger/issues/671

[w=1][y=1][p=0.8233] The problem in the first case is that Dagger is overeager to use a single static factory instance for generating instances of scoped generic classes.

[w=1][y=1][p=0.7090] However, when adding another @Inject element or removing scoping it succeeds:

[w=0][y=1][p=0.6657] Scoped generic class fails to build

--------------------

Permission Denial when trying to access contacts in Android
https://github.com/morenoh149/react-native-contacts/issues/516

[w=0][y=1][p=0.7677] You must use read profile permission in android platform.

[w=0][y=1][p=0.7082] If permissions are not granted, the callback should be called, with the error field being non-null/undefined.

[w=0][y=1][p=0.6797] In iOS, permissions aren't granted, it will be handled in the error block:

[w=0][y=1][p=0.6111] The weirdest part was how getAll did not throw an error, it just hung indefintiely,

In [23]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
examples_per_source_type(source_type='so', n_samples=4)

SO

Why settings.xml layout is overlapping the ActionBar/Toolbar?
https://stackoverflow.com/questions/29738510

[w=0][y=1][p=0.8031] Must be a boolean value, either `` true'' or `` false''.

[w=1][y=1][p=0.7809] If true, adjusts the padding of this view to leave space for the system windows.

[w=1][y=1][p=0.7795] internal attribute to adjust view layout based on system windows such as the status bar.

[w=3][y=1][p=0.6278] Use android: fitsSystemWindows = `` true'' in the root view of your layout -LRB- LinearLayout in your case -RRB-.

[w=0][y=1][p=0.6191] Make sure you don't have the following line as AS puts it by default:

--------------------

Android: rotate canvas around the center of the screen
https://stackoverflow.com/questions/8712652

[w=3][y=1][p=0.9036] You can also do it by rotating the canvas before drawing:

[w=2][y=1][p=0.8902] You can either rotate your bitmap when you draw it by using a matrix:

[w=1][y=1][p=0.8758] You have to rotate the canvas first and then draw wh

In [24]:
#@title Sample prediction outputs for MISC sources

logger.info(Fore.RED + "MISC" + Style.RESET_ALL)
examples_per_source_type(source_type='misc', n_samples=4)

MISC

Android: rotate canvas around the center of the screen
https://dzone.com/articles/android-rotate-and-scale

[w=2][y=1][p=0.8965] that is done because rotation is around the top left point ( the origin ) of the view.

[w=1][y=1][p=0.8914] when you press the rotate button, each of the images is rotated around its center, while maintaining its position in the center of the region on the screen.

[w=0][y=1][p=0.8659] first, the matrix object is set so it will fit the bitmap into the rectangle for the view.

[w=1][y=1][p=0.8601] the second step is to translate the view up and left by half the width and half the height.

[w=0][y=1][p=0.8480] the offset cycles you through four different offsets.

--------------------

Is there an accepted best-practice on making asynchronous HTTP requests in Android?
https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java

[w=0][y=1][p=0.8788] It also has the option to make requests synchronously or asynchronously by using the CompletableFuture